## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-12-19-14-53-15 +0000,bbc,Young people will feel burden of UK's ageing s...,https://www.bbc.com/news/articles/c997l3elxmxo...
1,2025-12-19-14-48-49 +0000,wapo,H-1B workers flew to India to renew U.S. visas...,https://www.washingtonpost.com/world/2025/12/1...
2,2025-12-19-14-48-06 +0000,nyt,Europe Funnels Billions to Ukraine but Wobbles...,https://www.nytimes.com/2025/12/19/world/europ...
3,2025-12-19-14-47-30 +0000,nyt,Portuguese Colleagues Laud Slain MIT Professor...,https://www.nytimes.com/2025/12/19/us/mit-prof...
4,2025-12-19-14-44-18 +0000,nypost,Trump just legalized marijuana nationwide — he...,https://nypost.com/2025/12/19/business/trump-j...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2390/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
32,trump,53
101,new,26
47,bondi,16
140,deal,16
41,billion,15


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
237,2025-12-18-23-26-52 +0000,nypost,GOP senators push Trump to use $5 billion in R...,https://nypost.com/2025/12/18/world-news/gop-s...,127
289,2025-12-18-21-08-55 +0000,nyt,"Trump Media to Merge With TAE Technologies, a ...",https://www.nytimes.com/2025/12/18/business/tr...,118
282,2025-12-18-21-37-00 +0000,wsj,Trump Media to Merge With Nuclear Fusion Firm ...,https://www.wsj.com/business/deals/trump-media...,115
74,2025-12-19-11-43-00 +0000,wsj,The attempted renaming of the Kennedy Center m...,https://www.wsj.com/politics/policy/with-trump...,107
257,2025-12-18-22-28-39 +0000,nypost,President Trump gives federal workers 2 new ho...,https://nypost.com/2025/12/18/us-news/presiden...,104


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
237,127,2025-12-18-23-26-52 +0000,nypost,GOP senators push Trump to use $5 billion in R...,https://nypost.com/2025/12/18/world-news/gop-s...
293,92,2025-12-18-20-58-14 +0000,nypost,"New York Times columnist David Brooks, Google ...",https://nypost.com/2025/12/18/us-news/david-br...
336,76,2025-12-18-18-15-56 +0000,bbc,Faisal Islam: Will pre-Christmas interest rate...,https://www.bbc.com/news/articles/c1kp2722399o...
107,71,2025-12-19-10-04-01 +0000,bbc,Brown University and MIT professor shooting su...,https://www.bbc.com/news/articles/cx2e14z2j9jo...
103,56,2025-12-19-10-30-00 +0000,wsj,Affordability means not just the prices we pay...,https://www.wsj.com/economy/jobs-could-soon-re...
126,54,2025-12-19-09-37-25 +0000,wapo,Australia announces plan to buy back guns afte...,https://www.washingtonpost.com/world/2025/12/1...
289,50,2025-12-18-21-08-55 +0000,nyt,"Trump Media to Merge With TAE Technologies, a ...",https://www.nytimes.com/2025/12/18/business/tr...
204,47,2025-12-19-01-19-33 +0000,bbc,Kennedy Center to be renamed Trump-Kennedy Cen...,https://www.bbc.com/news/articles/cp84pxvp87eo...
11,43,2025-12-19-14-18-07 +0000,nypost,NASCAR driver Greg Biffle’s emotional last pos...,https://nypost.com/2025/12/19/sports/nascar-dr...
243,38,2025-12-18-23-07-00 +0000,wsj,President Trump directed the federal governmen...,https://www.wsj.com/politics/policy/marijuana-...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
